<p style="float: left;"><a href="nested-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="currying.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Higher-order Functions

Higher-order functions are functions that either take other functions as parameters or return a function as a result. 

<span style="color:black">**In Scala, functions and methods are _first-class values_**.</span> In simple terms, this means that <span style="color:black">**functions can be passed around just like any other value**</span>, such as integers or sequences.

<span style="color:black">**The term _higher-order function_ applies to both methods and functions**</span> that accept functions as arguments or return functions as results.

A common example of a higher-order function in Scala is `map`, which is available for collections.

In [1]:
val salaries = Seq(20000, 70000, 40000)
val doubleSalary = (x: Int) => x * 2
val newSalaries = salaries.map(doubleSalary) // List(40000, 140000, 80000)

salaries: Seq[Int] = List(20000, 70000, 40000)
doubleSalary: Int => Int = ammonite.$sess.cmd1$Helper$$Lambda$2073/1243281061@4c4e77ba
newSalaries: Seq[Int] = List(40000, 140000, 80000)

**`doubleSalary`** is a function which takes a single **`x: Int`**, and returns **x * 2**. 

To shrink the code, **we could make the function _anonymous_** and pass it directly as
an argument to `map`:

In [2]:
val salaries = Seq(20000, 70000, 40000)
val newSalaries = salaries.map(x => x * 2) // List(40000, 140000, 80000)

salaries: Seq[Int] = List(20000, 70000, 40000)
newSalaries: Seq[Int] = List(40000, 140000, 80000)

Notice how `x` is not declared as an `Int` in the above example. That's because the
compiler can infer the type based on the type of function map expects. 

An even more idiomatic way to write the same piece of code would be:

In [3]:
val salaries = Seq(20000, 70000, 40000)
val newSalaries = salaries.map(_ * 2)

salaries: Seq[Int] = List(20000, 70000, 40000)
newSalaries: Seq[Int] = List(40000, 140000, 80000)

## Coercing methods into functions

It is also possible to pass methods as arguments to higher-order functions because
the Scala compiler will coerce the method into a function via _eta-expansion_.

In [4]:
case class WeeklyWeatherForecast(temperatures: Seq[Double]) {

  private def convertCtoF(temp: Double) = temp * 1.8 + 32

  def forecastInFahrenheit: Seq[Double] = temperatures.map(convertCtoF) // <-- passing the method convertCtoF
}

defined class WeeklyWeatherForecast

Here the method `convertCtoF` is passed to `forecastInFahrenheit`. **This is possible because the compiler coerces `convertCtoF` to the function `x => convertCtoF(x)`.**

## Functions that accept functions

One reason to use higher-order functions is to reduce redundant code. 

Let's say you wanted some methods that could raise someone's salaries by various factors. Without creating a higher-order function.

In [4]:
object SalaryRaiser {
  def smallPromotion(salaries: List[Double]): List[Double] =
    salaries.map(salary => salary * 1.1)

  def greatPromotion(salaries: List[Double]): List[Double] =
    salaries.map(salary => salary * math.log(salary))

  def hugePromotion(salaries: List[Double]): List[Double] =
    salaries.map(salary => salary * salary)
}

defined object SalaryRaiser

<span style="color:red">**Notice how each of the three methods vary only by the multiplication factor.**</span>

To simplify, you can extract the repeated code into a higher-order function like so:

In [2]:
object SalaryRaiser {
  private def promotion(salaries: List[Double], promotionFn: Double => Double): List[Double] = // <- private
    salaries.map(promotionFn)

  def smallPromotion(salaries: List[Double]): List[Double] =
    promotion(salaries, salary => salary * 1.1)

  def bigPromotion(salaries: List[Double]): List[Double] =
    promotion(salaries, salary => salary * math.log(salary))

  def hugePromotion(salaries: List[Double]): List[Double] =
    promotion(salaries, salary => salary * salary)
}

defined object SalaryRaiser

The new method, **`promotion`**, takes the salaries of type **`List[Double]`** plus a function of type **`Double => Double`**.

## Functions that return functions

There are certain cases where you want to generate a function. Here's an example
of a method that returns a function.

In [3]:
def urlBuilder(ssl: Boolean, domainName: String): (String, String) => String =
  val schema = if (ssl) "https://" else "http://"
  (endpoint: String, query: String) => s"$schema$domainName/$endpoint?$query"

val domainName = "www.example.com"
def getURL = urlBuilder(ssl=true, domainName) 
val endpoint = "users"
val query = "id=1"
val url = getURL(endpoint, query) // "https://www.example.com/users?id=1": String

defined function urlBuilder
domainName: String = "www.example.com"
defined function getURL
endpoint: String = "users"
query: String = "id=1"
url: String = "https://www.example.com/users?id=1"

<span style="color:black">The method **`urlBuilder`** returns a function of type **`(String, String) => String`**.</span>

<p style="float: left;"><a href="nested-methods.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="currying.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>